In [1]:
import numpy as np
import random
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
from scipy.spatial.distance import cdist
from itertools import combinations
from numpy.lib.stride_tricks import sliding_window_view
from time import time

In [2]:
def calculate_average(*args,**kwargs):
    return np.mean(args)

In [3]:
def calculate_cost(data: np.ndarray) -> np.ndarray:
    costs = cdist(data[:,2].reshape(-1,1), data[:,2].reshape(-1,1),calculate_average)
    distances = np.round(cdist(data[:,:2], data[:,:2], 'euclidean'))
    distances = distances + costs
    return distances

In [4]:
def random_initial_solution(data: np.ndarray) -> np.ndarray:
    """
    Creates shuffled array of indices with size equal to half of the data points.
    :param data: Data points
    :type data: np.ndarray
    :return: Data indices of nodes contained in initial solutions
    :rtype: np.ndarray
    """
    indices = np.arange(len(data))
    np.random.shuffle(indices)
    order = indices[:len(data)//2]
    return order

In [5]:
def create_node_exchange_neighborhood(order: np.ndarray, num_of_all_points: int) -> np.ndarray:
    ind = np.arange(len(order))
    #Inter-route
    indices_for_all_data = np.arange(num_of_all_points)
    not_selected = np.setdiff1d(indices_for_all_data, order)
    XX, YY = np.meshgrid(ind,not_selected)
    inter_route = np.dstack([XX,YY]).reshape(-1, 2)
    is_inter_route = np.zeros((inter_route.shape[0],1),dtype=np.int32)
    inter_route = np.concatenate((is_inter_route,inter_route),1)
    return inter_route

In [6]:
def create_neighborhood(order: np.ndarray, num_of_all_points: int) -> np.ndarray:
    """
    Creates every possible move.
    ------------
    Inter-route
        Creates every possible combination of adding a new node to the nodes contained in the solution
    Edge exchange
        Creates every possible combination of pairs contained in the solution

    :param order: Array of indices contained in the solution
    :type order: np.ndarray
    :param num_of_all_points: Number of all data points
    :type num_of_all_points: int
    :return: Neighborhood with first column defining whether move is inter-route and the following columns defining move.
    :rtype:
    """
    ind = np.arange(len(order))
    #Inter-route
    inter_route = create_node_exchange_neighborhood(order,num_of_all_points)
    # #Edge exchange
    edge_ex = np.array(list(combinations(ind, 2)))
    is_inter_route = np.ones((edge_ex.shape[0],1),dtype=np.int32)
    edge_ex = np.concatenate((is_inter_route,edge_ex),1)
    # return edge_ex
    neighborhood = np.vstack((inter_route,edge_ex))
    return neighborhood
    # return edge_ex

In [7]:
def calculate_value(data: np.ndarray, distances: np.ndarray, order: np.ndarray) -> tuple:
    """
    :param data: Data points
    :type data: np.ndarray
    :param distances: Distance matrix
    :type distances: np.ndarray
    :param order: Array of indices contained in the solution
    :type order: np.ndarray
    :return: Tuple containing score for the current solution and data points corresponding to the solution
    :rtype:
    """
    order = np.append(order,order[0])
    x,y =sliding_window_view(order.reshape(1,-1),(1,2)).reshape(-1,2).T
    value = distances[x,y].sum()
    path = data[order]
    return value, path

In [8]:
def calculate_deltas_for_node_exchange_moves(order:np.ndarray, distances: np.ndarray, possible_inter_moves: np.ndarray) -> np.ndarray:

    left_neighbors = order[possible_inter_moves[:,1] -1]
    right_neighbors = order[np.remainder(possible_inter_moves[:,1] +1, len(order))]
    currently_chosen_nodes = order[possible_inter_moves[:,1]]
    current_distances = distances[left_neighbors,currently_chosen_nodes] + distances[currently_chosen_nodes,right_neighbors]

    possible_new_nodes = possible_inter_moves[:,2]
    possible_new_distances = distances[left_neighbors,possible_new_nodes] + distances[possible_new_nodes,right_neighbors]

    all_deltas = current_distances - possible_new_distances
    improving_moves_mask = all_deltas>0


    improving_deltas = all_deltas[improving_moves_mask].reshape(-1,1).astype(np.int32)

    improving_moves = possible_inter_moves[improving_moves_mask]
    improving_moves = np.concatenate((improving_deltas,improving_moves),axis=1)
    return improving_moves

In [9]:
def calculate_deltas_for_edge_exchange(order:np.ndarray, distances: np.ndarray, possible_edge_exchange_moves: np.ndarray) -> np.ndarray:
    left_nodes = order[possible_edge_exchange_moves[:,1]]
    left_nodes_predecessors = order[possible_edge_exchange_moves[:,1] -1]

    right_nodes = order[possible_edge_exchange_moves[:,2]]
    right_nodes_predecessors = order[possible_edge_exchange_moves[:,2] -1]

    current_distances = distances[left_nodes,left_nodes_predecessors] + distances[right_nodes,right_nodes_predecessors]
    distances_after_edge_swap = distances[left_nodes,right_nodes] + distances[left_nodes_predecessors,right_nodes_predecessors]

    all_deltas = current_distances - distances_after_edge_swap
    improving_moves_mask = all_deltas>0


    improving_deltas = all_deltas[improving_moves_mask].reshape(-1,1).astype(np.int32)
    improving_moves = possible_edge_exchange_moves[improving_moves_mask]
    improving_moves = np.concatenate((improving_deltas,improving_moves),axis=1)
    return improving_moves

In [10]:
def calculate_deltas(order: np.ndarray, distances: np.ndarray, neighborhood: np.ndarray) -> np.ndarray:
    possible_inter_moves = neighborhood[neighborhood[:,0] == 0]
    improving_inter_moves = calculate_deltas_for_node_exchange_moves(order=order,
                                                                   distances=distances,
                                                                   possible_inter_moves=possible_inter_moves)

    possible_edge_exchange_moves = neighborhood[neighborhood[:,0] != 0]

    improving_edge_exchange_moves = calculate_deltas_for_edge_exchange(order=order,
                                                                       distances=distances,
                                                                       possible_edge_exchange_moves=possible_edge_exchange_moves)

    all_improving_moves = np.concatenate((improving_inter_moves,improving_edge_exchange_moves),axis=0)

    sorted_improving_moves = all_improving_moves[all_improving_moves[:,0].argsort()[::-1]]

    return sorted_improving_moves

In [11]:
def remove_not_applicable_moves(deltas:np.ndarray,best_move: np.ndarray) -> np.ndarray:
    node_exchange_moves = deltas[:,1] == 0
    edge_exchange_moves = deltas[:,1] == 1

    cond = np.bitwise_and(node_exchange_moves,deltas[:,3] ==  best_move[3])

    not_applicable_moves = cond
    return deltas[~not_applicable_moves]

In [12]:
def update_order_node_exchange(order:np.ndarray, best_move: np.ndarray) -> np.ndarray:
    order[best_move[2]] = best_move[3]
    return order

In [13]:
def update_order_edge_exchange(order:np.ndarray, best_move: np.ndarray) -> np.ndarray:
    ind = sorted([best_move[2], best_move[3]])
    if ind[0] == 0:
        order = np.concatenate((order[ind[1]-1::-1],order[ind[1]:]))
    else:
        order = np.concatenate((order[:ind[0]],order[ind[1]-1:ind[0]-1:-1],order[ind[1]:]))
    return order

In [14]:
def get_moves_to_recalculate_node_exchange(order:np.ndarray,deltas: np.ndarray, best_move: np.ndarray) -> np.ndarray:
    node_exchange_moves = deltas[:,1] == 0
    edge_exchange_moves = deltas[:,1] == 1

    cond1 = np.bitwise_and(node_exchange_moves,deltas[:,2] == best_move[2])
    cond2 = np.bitwise_and(node_exchange_moves,deltas[:,2] == (best_move[2] - 1) % len(order))
    cond3 = np.bitwise_and(node_exchange_moves,deltas[:,3] == best_move[3])
    cond4 = np.bitwise_and(node_exchange_moves,deltas[:,2] == (best_move[2] +1) % len(order))

    cond5 = np.bitwise_and(edge_exchange_moves,deltas[:,2] == best_move[2])
    cond6 = np.bitwise_and(edge_exchange_moves,deltas[:,2] == (best_move[2] +1) % len(order))
    cond9 = np.bitwise_and(edge_exchange_moves,deltas[:,3] == best_move[2])
    cond10 = np.bitwise_and(edge_exchange_moves,deltas[:,3] == (best_move[2] +1) % len(order))

    moves_to_remove = cond1 | cond2 | cond3 | cond4 | cond5 | cond6 | cond9 | cond10
    return moves_to_remove

In [15]:
def create_new_moves(order: np.ndarray, removed_node: int) -> np.ndarray:
    ind = np.arange(len(order))
    removed_node = np.array([removed_node])
    XX, YY = np.meshgrid(ind,removed_node)
    inter_route = np.dstack([XX,YY]).reshape(-1, 2)
    is_inter_route = np.zeros((inter_route.shape[0],1),dtype=np.int32)
    inter_route = np.concatenate((is_inter_route,inter_route),1)
    return inter_route

In [16]:
def get_moves_to_recalculate_edge_exchange(deltas: np.ndarray, best_move: np.ndarray) -> np.ndarray:
    node_exchange_moves = deltas[:,1] == 0

    cond1 = np.bitwise_and(node_exchange_moves,deltas[:,2] == best_move[2])
    cond2 = np.bitwise_and(node_exchange_moves,deltas[:,2] == (best_move[2] -1) % len(order))
    cond3 = np.bitwise_and(node_exchange_moves,deltas[:,2] == (best_move[3] -1) % len(order))
    cond4 = np.bitwise_and(node_exchange_moves,deltas[:,2] == best_move[3])

    moves_to_remove = cond1 | cond2 | cond3 | cond4
    return moves_to_remove

In [17]:
def local_search_steepest(order, distances):
    neighborhood = create_neighborhood(order=order,num_of_all_points=len(data))
    deltas = calculate_deltas(order,distances,neighborhood)
    counter =0

    while len(deltas) > 0:
        counter +=1
        best_move = deltas[0]
        # print("best_move: ",best_move)
        if best_move[1] == 0:
            deltas = remove_not_applicable_moves(deltas,best_move)
            new_deltas = create_new_moves(order,order[best_move[2]])

            order = update_order_node_exchange(order,best_move=best_move)
            moves_to_recalculate = get_moves_to_recalculate_node_exchange(order,deltas, best_move)

            # print("node exchange to recalculate: \n",deltas[moves_to_recalculate])
            nodes_moves = deltas[:,1] ==0

            deltas_to_recalculate = np.concatenate((deltas[moves_to_recalculate,1:],new_deltas),axis=0)
            updated_deltas = calculate_deltas(order, distances, deltas_to_recalculate)
            # print("node exchange after recalculate: \n",updated_deltas)

            deltas = np.concatenate((deltas[~moves_to_recalculate],updated_deltas),axis=0)
            deltas = deltas[deltas[:,0].argsort()[::-1]]
            # print("all deltas \n",deltas[:5])

        elif best_move[1] == 1:
            order = update_order_edge_exchange(order,best_move)
            updated_deltas = calculate_deltas(order,distances,neighborhood[neighborhood[:,0] ==1])
            new_deltas = calculate_deltas(order,distances,create_node_exchange_neighborhood(order,len(data)))

            deltas = np.concatenate((new_deltas,updated_deltas),axis=0)
            deltas = deltas[deltas[:,0].argsort()[::-1]]
    return order

In [18]:
def destroy(order):
    to_delete = set(random.sample(range(len(order)),len(order) // 4))
    return [x for i,x in enumerate(order) if not i in to_delete]

In [19]:
def repair(order, distances):
    # greedy cycle
    return order

In [20]:
def LSNS(data, distances, args=dict(running_time=10, local_search=True)):
    best_solution = (9999999999, None)
    order = random_initial_solution(data)
    if args["local_search"]:
        order = local_search_steepest(order, distances)
    best_order = order
    start = time()
    count = 0
    while(args["running_time"] > time() - start):
        count += 1
        order = destroy(best_order)
        order = repair(order, distances) #ten greedy cycle
        assert len(best_order) == len(order), f"Length of best_order is {len(best_order)} but length of order is {len(order)}"
        if args["local_search"]:
            order = local_search_steepest(order, distances)
        solution = calculate_value(data, distances, order)
        if solution[0] < best_solution[0]:
            best_solution = solution
            best_order = order
#         best_solution = min((best_solution, calculate_value(data, distances, order)), key = lambda t: t[0])
    return best_solution, count

In [21]:
def evaluate(func, data, n=20, args=None):
    print(func, args)
    distances = calculate_cost(data)
    total, worst_value, best_value, best_solution = 0, 0, float('inf'), None
    total_time, worst_time, best_time = 0,0,float('inf')
    total_iterations, most_iterations, least_iterations = 0,0,float('inf')
    for _ in tqdm(range(n)):
        start = time()
        (value, solution), iterations = func(data, distances, args)
        elapsed = time() - start
        total_time += elapsed
        if elapsed < best_time:
            best_time = elapsed
        worst_time = max(worst_time, elapsed)


        total += value
        if value < best_value:
            best_solution = solution
            best_value = value
        worst_value = max(worst_value, value)
        
        total_iterations += iterations
        least_iterations = min(least_iterations, iterations)
        most_iterations = max(most_iterations, iterations)

    return dict(average_score=total / n,
                worst_score=worst_value,
                best_score=best_value,
                solution=np.array(best_solution).T,
                average_time=total_time / n,
                worst_time= worst_time,
                best_time = best_time,
                average_iterations=total_iterations / n,
                most_iterations=most_iterations,
                least_iterations=least_iterations)

In [22]:
def summarize(func, data, title, args):
    evaluation_results = evaluate(func=func,
                                    data=data,
                                     n = 20,
                                     args = args)
    (x, y, z) = evaluation_results["solution"]
    print(f'{title}:')
    print(f'average time: {evaluation_results["average_time"]}\tworst time: {evaluation_results["worst_time"]}\tbest time: {evaluation_results["best_time"]}')
    print(f'average iterations: {evaluation_results["average_iterations"]}\tmost iterations: {evaluation_results["most_iterations"]}\tleast iterations: {evaluation_results["least_iterations"]}')
    print(f'average score: {evaluation_results["average_score"]}\tworst score: {evaluation_results["worst_score"]}\tbest score: {evaluation_results["best_score"]}')
    print('Best solution:')

    plt.figure(figsize=(15,10))
    plt.scatter(data[:,0], data[:,1], np.full(data[:,0].size, 100), data[:,2], cmap='gray_r')
    plt.clim(0, 2000)
    plt.colorbar().set_label('cost', labelpad=-45, y=1.025, rotation=0)
    plt.plot(x, y, 'r')
    plt.xlim([-100, 4100])
    plt.ylim([-100, 2100])
    plt.title(title)
    plt.show()
    return evaluation_results

In [23]:
data = np.loadtxt('TSPC.csv', dtype=int, delimiter=';')
running_time = 15.91

In [24]:
args = dict(running_time=running_time, local_search=True)
_ = summarize(LSNS, data, 'LSNS', args)

<function LSNS at 0x7fa63dd2f7b8> {'running_time': 15.91, 'local_search': True}


AssertionError: Length of best_order is 100 but length of order is 75

In [ ]:
data = np.loadtxt('TSPD.csv', dtype=int, delimiter=';')
running_time = 15.83

In [ ]:
args = dict(running_time=1, local_search=True)
_ = summarize(LSNS, data, 'LSNS', args)